In [1]:
import ee

# Initialize Earth Engine
ee.Initialize()

def PCA(masked_image, scale, region):
    image = masked_image.unmask()
    band_names = image.bandNames()
    
    # Mean center the data to enable a faster covariance reducer and an SD stretch of the principal components.
    mean_dict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=scale,
        maxPixels=1e13,
        tileScale=16
    )
    
    means = ee.Image.constant(mean_dict.values(band_names))
    centered = image.subtract(means)
    
    # Helper function to return a list of new band names.
    def get_new_band_names(prefix):
        seq = ee.List.sequence(1, band_names.size())
        return seq.map(lambda b: ee.String(prefix).cat(ee.Number(b).int()))
    
    # Function to perform PCA and return the principal components.
    def get_principal_components(centered, scale, region):
        # Collapse the bands of the image into a 1D array per pixel.
        arrays = centered.toArray()
        
        # Compute the covariance of the bands within the region.
        covar = arrays.reduceRegion(
            reducer=ee.Reducer.centeredCovariance(),
            geometry=region,
            scale=scale,
            maxPixels=1e13,
            tileScale=16
        )
        
        # Get the 'array' covariance result and cast to an array.
        covar_array = ee.Array(covar.get('array'))
        
        # Perform an eigen analysis and slice apart the values and vectors.
        eigens = covar_array.eigen()
        
        # This is a P-length vector of Eigenvalues.
        eigen_values = eigens.slice(1, 0, 1)
        
        # Compute Percentage Variance of each component
        eigen_values_list = eigen_values.toList().flatten()
        total = eigen_values_list.reduce(ee.Reducer.sum())
        
        percentage_variance = eigen_values_list.map(
            lambda item: ee.List([
                eigen_values_list.indexOf(item).add(1).format('%02d'),
                ee.Number(item).divide(total).multiply(100).format('%.2f')
            ])
        )
        
        # Create a dictionary to set properties on final image
        variance_dict = ee.Dictionary(percentage_variance.flatten())
        
        # This is a PxP matrix with eigenvectors in rows.
        eigen_vectors = eigens.slice(1, 1)
        
        # Convert the array image to 2D arrays for matrix computations.
        array_image = arrays.toArray(1)
        
        # Left multiply the image array by the matrix of eigenvectors.
        principal_components = ee.Image(eigen_vectors).matrixMultiply(array_image)
        
        # Turn the square roots of the Eigenvalues into a P-band image.
        # Call abs() to turn negative eigenvalues to positive before taking the square root.
        sd_image = ee.Image(eigen_values.abs().sqrt()).arrayProject([0]).arrayFlatten([get_new_band_names('sd')])
        
        # Turn the PCs into a P-band image, normalized by SD.
        return principal_components\
            .arrayProject([0])\
            .arrayFlatten([get_new_band_names('pc')])\
            .divide(sd_image)\
            .set(variance_dict)
    
    pc_image = get_principal_components(centered, scale, region)
    return pc_image.mask(masked_image.mask())

# Example usage:
composite = ee.Image("users/ujavalgandhi/e2e/arkavathy_2019_composite")
boundary = ee.FeatureCollection("users/ujavalgandhi/e2e/arkavathy_boundary")
geometry = boundary.geometry()
scale = 20

# Run the PCA function
pca_result = PCA(composite, scale, geometry)

# Extract the properties of the PCA image
variance = pca_result.toDictionary()
print('Variance of Principal Components:', variance.getInfo())

# Select the first 3 principal components
pca_bands = pca_result.select(['pc1', 'pc2', 'pc3'])
print('First 3 PCA Bands:', pca_bands.getInfo())


Variance of Principal Components: {'01': '69.60', '02': '22.53', '03': '5.38', '04': '0.90', '05': '0.58', '06': '0.39', '07': '0.27', '08': '0.16', '09': '0.09', '10': '0.07', '11': '0.03', '12': '0.02'}
First 3 PCA Bands: {'type': 'Image', 'bands': [{'id': 'pc1', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [5428, 12389], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 77.20409012614085, 0, -8.983152841195215e-05, 13.383370597397867]}, {'id': 'pc2', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [5428, 12389], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 77.20409012614085, 0, -8.983152841195215e-05, 13.383370597397867]}, {'id': 'pc3', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [5428, 12389], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 77.20409012614085, 0, -8.983152841195215e-05, 13.383370597397867]}], 'properties': {'04': '0.90', '05': '0.58', '0